In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("RetailSalesAnalytics") \
    .getOrCreate()


In [ ]:
# Adjust file path and format as needed (e.g., CSV, Parquet)
sales_df = spark.read.csv("data/Online_Retail.csv", header=True, inferSchema=True)


In [ ]:
review_df = spark.read.csv("data/Online_Retail.csv", header=True, inferSchema=True)


In [ ]:
# Inspect first few rows
sales_df.show(5)
review_df.show(5)

# Check schema
sales_df.printSchema()
review_df.printSchema()

# Count rows
print("Sales Dataset Rows:", sales_df.count())
print("Reviews Dataset Rows:", review_df.count())


+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|01-12-2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 08:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
only showing top 5 rows

+---------+---------+---------

In [ ]:
# Summary of numeric columns
sales_df.describe().show()
review_df.describe().show()


+-------+------------------+------------------+--------------------+-----------------+----------------+-----------------+------------------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|         Quantity|     InvoiceDate|        UnitPrice|        CustomerID|    Country|
+-------+------------------+------------------+--------------------+-----------------+----------------+-----------------+------------------+-----------+
|  count|            541909|            541909|              540455|           541909|          541909|           541909|            406829|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0| 9.55224954743324|            NULL|4.611113626082965|15287.690570239585|       NULL|
| stddev|13428.417280799484| 16799.73762842768|                NULL|218.0811578502348|            NULL|96.75985306117944|1713.6003033215932|       NULL|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|           -80

In [ ]:
from pyspark.sql.functions import col

# Count missing values in each column
sales_df.select([col(c).isNull().alias(c) for c in sales_df.columns]).show()
review_df.select([col(c).isNull().alias(c) for c in review_df.columns]).show()


+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false| 

In [ ]:
# Count duplicate rows
print("Sales Duplicates:", sales_df.count() - sales_df.dropDuplicates().count())
print("Reviews Duplicates:", review_df.count() - review_df.dropDuplicates().count())


Sales Duplicates: 5268
Reviews Duplicates: 5268


In [ ]:
## Example: Fill missing customer IDs with "unknown"
sales_df = sales_df.fillna({"CustomerID": "unknown"})

# Drop rows with critical missing fields, assuming 'InvoiceNo' and 'StockCode'
# are the intended critical fields based on available columns.
review_df = review_df.dropna(subset=["InvoiceNo", "StockCode"])


In [ ]:
sales_df = sales_df.dropDuplicates()
review_df = review_df.dropDuplicates()


In [ ]:
from pyspark.sql.functions import to_date, col

# Convert string dates to proper date format
sales_df = sales_df.withColumn("InvoiceDate", to_date(col("InvoiceDate"), "MM/dd/yyyy"))
# Assuming you want to convert 'InvoiceDate' in review_df as well, since 'review_date' doesn't exist
review_df = review_df.withColumn("InvoiceDate", to_date(col("InvoiceDate"), "MM/dd/yyyy"))
# If you have a column with a different name containing the review date,
# replace 'InvoiceDate' with that column name in the line above.
# For example, if the column is named 'ReviewDate', use:
# review_df = review_df.withColumn("ReviewDate", to_date(col("ReviewDate"), "yyyy-MM-dd"))

In [ ]:
# Re-check for nulls and duplicates
sales_df.select([col(c).isNull().alias(c) for c in sales_df.columns]).show()
review_df.select([col(c).isNull().alias(c) for c in review_df.columns]).show()

print("Sales Duplicates After Cleansing:", sales_df.count() - sales_df.dropDuplicates().count())
print("Reviews Duplicates After Cleansing:", review_df.count() - review_df.dropDuplicates().count())


+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|    false|    false|      false|   false|       true|    false|     false|  false|
|    false|    false|      false|   false|       true|    false|     false|  false|
|    false|    false|      false|   false|       true|    false|     false|  false|
|    false|    false|      false|   false|       true|    false|     false|  false|
|    false|    false|      false|   false|       true|    false|     false|  false|
|    false|    false|      false|   false|       true|    false|     false|  false|
|    false|    false|      false|   false|       true|    false|     false|  false|
|    false|    false|      false|   false|       true|    false|     false|  false|
|    false|    false|      false|   false|       true|    false|     false| 

In [ ]:
# Example: Top products by sales
sales_df.groupBy("StockCode").sum("Quantity").orderBy("sum(Quantity)", ascending=False).show()

# Example: Review counts by product
review_df.groupBy("StockCode").count().orderBy("count", ascending=False).show()


+---------+-------------+
|StockCode|sum(Quantity)|
+---------+-------------+
|    22197|        56427|
|    84077|        53751|
|   85099B|        47260|
|   85123A|        38811|
|    84879|        36122|
|    21212|        36016|
|    23084|        30597|
|    22492|        26437|
|    22616|        26299|
|    21977|        24719|
|    22178|        23825|
|    17003|        23017|
|    15036|        22552|
|    21915|        22036|
|    22386|        20992|
|    23203|        19995|
|    22086|        18876|
|    84946|        18874|
|    20725|        18858|
|    84991|        18015|
+---------+-------------+
only showing top 20 rows

+---------+-----+
|StockCode|count|
+---------+-----+
|   85123A| 2301|
|    22423| 2192|
|   85099B| 2156|
|    47566| 1720|
|    20725| 1626|
|    84879| 1489|
|    22720| 1469|
|    22197| 1468|
|    21212| 1367|
|    22383| 1328|
|    20727| 1323|
|    22457| 1272|
|    23203| 1260|
|     POST| 1256|
|    22386| 1245|
|    22469| 1232|
|    229